In [26]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from nba_api.stats.endpoints import leaguedashteamstats
import re

In [21]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_predict

# 1. Getting team stats for all regular seasons from 2015-2025

In [8]:
seasons = [f"{year}-{str(year+1)[-2:]}" for year in range(2015, 2025)]


season_data_frames = {}

for season in seasons:
    df = leaguedashteamstats.LeagueDashTeamStats(
        season=season,
        per_mode_detailed='PerGame'
    ).get_data_frames()[0]
    season_data_frames[season] = df

df_2015_2016 = season_data_frames['2015-16']
sorted_pts = df_2015_2016.sort_values('REB_RANK')
sorted_pts.tail()


,TEAM_ID,TEAM_NAME,GP,W,L,W_PCT,MIN,FGM,FGA,FG_PCT,...,REB_RANK,AST_RANK,TOV_RANK,STL_RANK,BLK_RANK,BLKA_RANK,PF_RANK,PFD_RANK,PTS_RANK,PLUS_MINUS_RANK
29,1610612764,Washington Wizards,82,41,41,0.500,48.2,39.5,85.8,0.460,...,26,5,16,8,26,6,21,16,9,17
16,1610612749,Milwaukee Bucks,82,33,49,0.402,48.4,38.4,82.2,0.467,...,27,9,25,13,7,26,17,21,25,26
14,1610612763,Memphis Grizzlies,82,42,40,0.512,48.4,36.8,83.6,0.440,...,28,25,6,6,21,29,27,9,24,20
17,1610612750,Minnesota Timberwolves,82,29,53,0.354,48.5,37.7,81.3,0.464,...,29,8,23,14,19,17,18,7,15,24
22,1610612755,Philadelphia 76ers,82,10,72,0.122,48.3,36.2,84.0,0.431,...,30,20,29,10,3,28,25,25,29,30


# 2. Making sure each team name is consistent across dataframes

In [9]:
team_name_to_abbr = {
    "Atlanta Hawks": "ATL",
    "Boston Celtics": "BOS",
    "Brooklyn Nets": "BKN",
    "Charlotte Hornets": "CHO",
    "Chicago Bulls": "CHI",
    "Cleveland Cavaliers": "CLE",
    "Dallas Mavericks": "DAL",
    "Denver Nuggets": "DEN",
    "Detroit Pistons": "DET",
    "Golden State Warriors": "GSW",
    "Houston Rockets": "HOU",
    "Indiana Pacers": "IND",
    "LA Clippers": "LAC",
    "Los Angeles Lakers": "LAL",
    "Memphis Grizzlies": "MEM",
    "Miami Heat": "MIA",
    "Milwaukee Bucks": "MIL",
    "Minnesota Timberwolves": "MIN",
    "New Orleans Pelicans": "NOP",
    "New York Knicks": "NYK",
    "Oklahoma City Thunder": "OKC",
    "Orlando Magic": "ORL",
    "Philadelphia 76ers": "PHI",
    "Phoenix Suns": "PHX",
    "Portland Trail Blazers": "POR",
    "Sacramento Kings": "SAC",
    "San Antonio Spurs": "SAS",
    "Toronto Raptors": "TOR",
    "Utah Jazz": "UTA",
    "Washington Wizards": "WAS"
}

for season, df in season_data_frames.items():
    df['TEAM_ABBREVIATION'] = df['TEAM_NAME'].map(team_name_to_abbr)
    df = df.rename(columns = {"TEAM_ABBREVIATION": "Team"}).drop(columns = ["TEAM_ID", "MIN", "GP", "W", "L"])
    season_data_frames[season] = df

df_2017_2018 = season_data_frames['2017-18']
df_2017_2018.head()





,TEAM_NAME,W_PCT,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,...,AST_RANK,TOV_RANK,STL_RANK,BLK_RANK,BLKA_RANK,PF_RANK,PFD_RANK,PTS_RANK,PLUS_MINUS_RANK,Team
0,Atlanta Hawks,0.293,38.2,85.5,0.446,11.2,31.0,0.360,15.8,20.2,...,8,27,12,23,28,14,14,25,26,ATL
1,Boston Celtics,0.671,38.3,85.1,0.450,11.5,30.4,0.377,16.0,20.7,...,20,15,23,18,6,17,22,20,6,BOS
2,Brooklyn Nets,0.341,38.2,86.8,0.441,12.7,35.7,0.356,17.4,22.6,...,9,25,30,16,27,23,19,14,24,BKN
3,Charlotte Hornets,0.439,39.0,86.7,0.450,10.0,27.2,0.369,20.2,27.0,...,24,3,28,18,20,2,1,10,17,CHO
4,Chicago Bulls,0.329,38.7,88.8,0.435,11.0,31.1,0.355,14.6,19.2,...,10,13,18,30,24,9,29,26,29,CHI


In [10]:
playoff_results = pd.read_csv('data/playoff_results.csv') 
playoff_results.tail()

,Yr,Lg,Series,Unnamed: 3,Unnamed: 4,Team,W,Unnamed: 7,Team.1,W.1,Unnamed: 10,Favorite,Underdog
130,2016,NBA,Western Conf Semifinals,May 1 - May 11 2016,NaN,Golden State Warriors (1),4,NaN,Portland Trail Blazers (5),1,NaN,GSW (-1600),POR (+1050)
131,2016,NBA,Western Conf Semifinals,Apr 30 - May 12 2016,NaN,Oklahoma City Thunder (3),4,NaN,San Antonio Spurs (2),2,NaN,SAS (-270),OKC (+230)
132,2016,NBA,Eastern Conf Finals,May 17 - May 27 2016,NaN,Cleveland Cavaliers (1),4,NaN,Toronto Raptors (2),2,NaN,CLE (-1350),TOR (+885)
133,2016,NBA,Western Conf Finals,May 16 - May 30 2016,NaN,Golden State Warriors (1),4,NaN,Oklahoma City Thunder (3),3,NaN,GSW (-430),OKC (+345)
134,2016,NBA,Finals,Jun 2 - Jun 19 2016,NaN,Cleveland Cavaliers (1),4,NaN,Golden State Warriors (1),3,NaN,GSW (-220),CLE (+180)


In [11]:
relabeled_dict = {
    "Team": "Team1",
    "Team.1": "Team2",
    "W": "Team1wins",
    "W.1": "Team2wins"
}

playoff_results = playoff_results.rename(columns=relabeled_dict)
playoff_results.head()

,Yr,Lg,Series,Unnamed: 3,Unnamed: 4,Team1,Team1wins,Unnamed: 7,Team2,Team2wins,Unnamed: 10,Favorite,Underdog
0,2024,NBA,Eastern Conf First Round,Apr 21 - May 1 2024,NaN,Boston Celtics (1),4,NaN,Miami Heat (8),1,NaN,BOS (-10000),MIA (+2000)
1,2024,NBA,Eastern Conf First Round,Apr 20 - May 5 2024,NaN,Cleveland Cavaliers (4),4,NaN,Orlando Magic (5),3,NaN,CLE (-210),ORL (+180)
2,2024,NBA,Eastern Conf First Round,Apr 21 - May 2 2024,NaN,Indiana Pacers (6),4,NaN,Milwaukee Bucks (3),2,NaN,IND (-125),MIL (+105)
3,2024,NBA,Eastern Conf First Round,Apr 20 - May 2 2024,NaN,New York Knicks (2),4,NaN,Philadelphia 76ers (7),2,NaN,NYK (-140),PHI (+120)
4,2024,NBA,Western Conf First Round,Apr 21 - Apr 29 2024,NaN,Oklahoma City Thunder (1),4,NaN,New Orleans Pelicans (8),0,NaN,OKC (-700),NOP (+500)


# 3. Get the 2 teams that played eachother in the playoff series

In [12]:
def get_teamID(team_name):
    cleaned_name = re.sub(r'\s\(\d+\)', '', team_name) # remove seed  & parentheses
    team_id = cleaned_name[:3].upper()     # get first three letters and convert to uppercase
    return team_id



playoff_results['Team1'] = playoff_results['Team1'].apply(get_teamID)
playoff_results['Team2'] = playoff_results['Team2'].apply(get_teamID) 
playoff_results.tail(15)

,Yr,Lg,Series,Unnamed: 3,Unnamed: 4,Team1,Team1wins,Unnamed: 7,Team2,Team2wins,Unnamed: 10,Favorite,Underdog
120,2016,NBA,Eastern Conf First Round,Apr 17 - Apr 24 2016,NaN,CLE,4,NaN,DET,0,NaN,CLE (-1700),DET (+1100)
121,2016,NBA,Eastern Conf First Round,Apr 16 - May 1 2016,NaN,TOR,4,NaN,IND,3,NaN,TOR (-400),IND (+325)
122,2016,NBA,Eastern Conf First Round,Apr 17 - May 1 2016,NaN,MIA,4,NaN,CHA,3,NaN,MIA (-170),CHO (+150)
123,2016,NBA,Eastern Conf First Round,Apr 16 - Apr 28 2016,NaN,ATL,4,NaN,BOS,2,NaN,ATL (-165),BOS (+145)
124,2016,NBA,Western Conf First Round,Apr 16 - Apr 27 2016,NaN,GOL,4,NaN,HOU,1,NaN,GSW (-10000),HOU (+4000)
125,2016,NBA,Western Conf First Round,Apr 17 - Apr 24 2016,NaN,SAN,4,NaN,MEM,0,NaN,SAS (-17000),MEM (+6625)
126,2016,NBA,Western Conf First Round,Apr 16 - Apr 25 2016,NaN,OKL,4,NaN,DAL,1,NaN,OKC (-3500),DAL (+1750)
127,2016,NBA,Western Conf First Round,Apr 17 - Apr 29 2016,NaN,POR,4,NaN,LOS,2,NaN,LAC (-525),POR (+415)
128,2016,NBA,Eastern Conf Semifinals,May 2 - May 8 2016,NaN,CLE,4,NaN,ATL,0,NaN,CLE (-500),ATL (+400)
129,2016,NBA,Eastern Conf Semifinals,May 3 - May 15 2016,NaN,TOR,4,NaN,MIA,3,NaN,TOR (-145),MIA (+125)


# 4. Better labeling

In [19]:
# changing team names to their actual abbreviations 
team_replacements = {
    'LOS': 'LAC',
    'CHA': 'CHO',
    'GOL': 'GSW',
    'BRO': 'BKN',
    'OKL': 'OKC',
    'NEW': 'NYK',
    'PHO': 'PHX'
}

playoff_results['Team1'] = playoff_results['Team1'].replace(team_replacements)
playoff_results['Team2'] = playoff_results['Team2'].replace(team_replacements)

print(playoff_results['Team1'].unique())


['BOS' 'CLE' 'IND' 'NYK' 'OKC' 'DAL' 'MIN' 'DEN' 'MIA' 'PHI' 'PHX' 'GSW'
 'LAC' 'MIL' 'MEM' 'BKN' 'UTA' 'ATL' 'TOR' 'HOU' 'POR' 'WAS' 'SAN']


# 5. Calculating each team's total wins in each playoffs using groupby

In [13]:
playoff_results["Year"] = playoff_results["Unnamed: 3"].str.extract(r"(\d{4})").astype(int)

team1_data = playoff_results[["Year", "Team1", "Team1wins"]].rename(columns={"Team1": "Team", "Team1wins": "Wins"})
team2_data = playoff_results[["Year", "Team2", "Team2wins"]].rename(columns={"Team2": "Team", "Team2wins": "Wins"})


all_teams = pd.concat([team1_data, team2_data])


team_wins_by_year = all_teams.groupby(["Year", "Team"], as_index=False)["Wins"].sum()
team_wins_by_year.sample(20)

,Year,Team,Wins
124,2023,SAC,3
12,2016,OKL,11
27,2017,POR,0
109,2022,UTA,2
67,2020,DEN,9
66,2020,DAL,2
42,2018,PHI,5
58,2019,ORL,1
130,2024,LOS,3
101,2022,MEM,6


In [14]:
merged_season_data = {}

seasons = ['2016-17', '2017-18', '2018-19', '2019-20', '2020-21', '2021-22', '2022-23', '2023-24']
for season in seasons:
    season_year = int(season.split('-')[1]) + 2000 # ex. "2021-22" ---> "2021" 
    playoff_wins_season = team_wins_by_year[team_wins_by_year['Year'] == int(season_year)]
    
    if not playoff_wins_season.empty:
        current_season = season_data_frames[season]
        merged_season = pd.merge(current_season, playoff_wins_season[['Team', 'Wins']], how='left', left_on='Team', right_on='Team')
        merged_season_data[season] = merged_season

        print(f"Combined data for {season} season:")
        print(merged_season.head())  
    else:
        print(f"No wins in {season}")

Combined data for 2016-17 season:
           TEAM_NAME  W_PCT   FGM   FGA  FG_PCT  FG3M  FG3A  FG3_PCT   FTM  \
0      Atlanta Hawks  0.524  38.1  84.4   0.451   8.9  26.1    0.341  18.1   
1     Boston Celtics  0.646  38.6  85.1   0.454  12.0  33.4    0.359  18.7   
2      Brooklyn Nets  0.244  37.8  85.2   0.444  10.7  31.6    0.338  19.4   
3  Charlotte Hornets  0.439  37.7  85.4   0.442  10.0  28.6    0.351  19.4   
4      Chicago Bulls  0.500  38.6  87.1   0.444   7.6  22.3    0.340  18.0   

    FTA  ...  TOV_RANK  STL_RANK  BLK_RANK  BLKA_RANK  PF_RANK  PFD_RANK  \
0  24.9  ...        28         7        14         22        6         3   
1  23.2  ...         8        18        23         23       21        11   
2  24.6  ...        29        21        18         30       25         9   
3  23.8  ...         1        27        17         28        1        15   
4  22.5  ...        12        15        16         12        2        25   

   PTS_RANK  PLUS_MINUS_RANK  Team  Wins

# 6. Finding most important features by calculating correlations

In [15]:
season_correlations = {}

for season, season_data in season_data_frames.items():
    season_year = int(season.split('-')[0]) + 1  # ex. "2016-17" -> "2017"
    playoff_wins_season = team_wins_by_year[team_wins_by_year['Year'] == season_year]
    df_merged = pd.merge(season_data, playoff_wins_season[['Team', 'Wins']], how='left', left_on='Team', right_on='Team')
    
    # only the numeric columns
    df_numeric = df_merged.select_dtypes(include=[float, int])

    # correlation matrix for numeric columns
    correlation_matrix = df_numeric.corr()

    # correlation of each feature with Playoff_Wins
    playoff_wins_correlation = correlation_matrix['Wins'].sort_values(ascending=False)

    # storing all the correlations
    season_correlations[season] = playoff_wins_correlation

for season, correlation_table in season_correlations.items():
    correlation_table = correlation_table.drop('Wins') 
    season_correlations[season] = correlation_table
    print(f"Correlation Table for {season}:")
    print(correlation_table)
    print("\n")



Correlation Table for 2015-16:
W_PCT              0.902001
PLUS_MINUS         0.811346
FG_PCT             0.631502
FG3_PCT            0.604005
STL_RANK           0.440587
DREB               0.374397
FGA_RANK           0.361779
FTM_RANK           0.336321
FTA_RANK           0.294568
MIN_RANK           0.248201
FG3M               0.205927
OREB_RANK          0.203645
FGM                0.188167
BLK                0.179996
PFD_RANK           0.161102
FT_PCT_RANK        0.137140
FT_PCT             0.123375
REB                0.123051
PTS                0.121259
FG3A               0.082866
AST               -0.032639
AST_RANK          -0.037616
FG3A_RANK         -0.055209
TOV_RANK          -0.124049
BLK_RANK          -0.145672
TOV               -0.152429
PFD               -0.183014
FG3M_RANK         -0.186498
FTM               -0.186884
REB_RANK          -0.204298
PTS_RANK          -0.204439
PF                -0.217808
BLKA              -0.224847
OREB              -0.225097
FTA              

In [16]:
for season, correlation_table in season_correlations.items():
  print(f"most important features for {season}:")
  print(correlation_table.head(7))
  print("\n")

most important features for 2015-16:
W_PCT         0.902001
PLUS_MINUS    0.811346
FG_PCT        0.631502
FG3_PCT       0.604005
STL_RANK      0.440587
DREB          0.374397
FGA_RANK      0.361779
Name: Wins, dtype: float64


most important features for 2016-17:
W_PCT         0.756282
BLK_RANK      0.727615
FG3M          0.665230
FG3A          0.621089
PLUS_MINUS    0.606113
PTS           0.541184
FGM           0.361434
Name: Wins, dtype: float64


most important features for 2017-18:
FG3M         0.703503
FG3A         0.652048
W_PCT        0.623442
FGA_RANK     0.519710
MIN_RANK     0.495541
OREB_RANK    0.488072
FG3_PCT      0.418843
Name: Wins, dtype: float64


most important features for 2018-19:
W_PCT         0.840052
PTS           0.762674
PLUS_MINUS    0.644062
FGM           0.625837
FT_PCT        0.610735
FGA           0.510660
FG_PCT        0.473357
Name: Wins, dtype: float64


most important features for 2019-20:
PFD           0.551002
TOV           0.480023
FTM           0.

# 7. Training and predicting!

In [20]:
def preprocess(combined_season_data, selected_features):
    combined_season_data = combined_season_data.fillna(0)  # replace null with 0
    
    # Target variable
    y_combined = combined_season_data['Wins']  #  'Wins' = target col
    X_combined = combined_season_data[selected_features]  # only most important features used in the model
    return X_combined, y_combined


def train(combined_season_data, selected_features):

    X_combined, y_combined = preprocess(combined_season_data, selected_features)
    
    # train test split
    X_train_combined, X_test_combined, y_train_combined, y_test_combined = train_test_split(
        X_combined, y_combined, test_size=0.2, random_state=42
    )
    
    # Standardizizin features
    scaler_combined = StandardScaler()
    X_train_scaled_combined = scaler_combined.fit_transform(X_train_combined)
    X_test_scaled_combined = scaler_combined.transform(X_test_combined)
    
    # Training
    model_combined = LinearRegression()
    model_combined.fit(X_train_scaled_combined, y_train_combined)
    
    # Evaluating
    y_pred_combined = model_combined.predict(X_test_scaled_combined)
    mse_combined = mean_squared_error(y_test_combined, y_pred_combined)
    print(f"Combined Model Performance (MSE): {mse_combined}")
    
    return model_combined, scaler_combined


all_season_data = pd.concat([merged_season_data[season] for season in seasons], axis=0)
selected_features_combined = ['W_PCT', 'PLUS_MINUS', 'FG3M', 'PTS', 'FG_PCT', 'AST', 'FG3_PCT', ]
combined_model_v1, combined_scaler_v1 = train(all_season_data, selected_features_combined)

# predictions
X_2024_25_combined = season_data_frames['2024-25'][selected_features_combined]
X_2024_25_scaled_combined = combined_scaler_v1.transform(X_2024_25_combined)  # sclaing to ensure each feature weighted the same 
predicted_wins_2024_25_combined = combined_model_v1.predict(X_2024_25_scaled_combined)
predicted_wins_2024_25_combined = [max(0, win) for win in predicted_wins_2024_25_combined] 

# Combine predictions with team names
predictions_combined = pd.DataFrame({
    'Team': season_data_frames['2024-25']['TEAM_NAME'],
    'Predicted Playoff Wins': predicted_wins_2024_25_combined
})


print(predictions_combined)


Combined Model Performance (MSE): 6.923877949545084
                      Team  Predicted Playoff Wins
0            Atlanta Hawks                1.474536
1           Boston Celtics                6.884206
2            Brooklyn Nets                0.866749
3        Charlotte Hornets                0.000000
4            Chicago Bulls                2.129802
5      Cleveland Cavaliers                7.434189
6         Dallas Mavericks                2.574842
7           Denver Nuggets                2.643726
8          Detroit Pistons                2.663495
9    Golden State Warriors                3.139369
10         Houston Rockets                4.418677
11          Indiana Pacers                3.476440
12             LA Clippers                3.496544
13      Los Angeles Lakers                4.111509
14       Memphis Grizzlies                4.026898
15              Miami Heat                3.066056
16         Milwaukee Bucks                3.520671
17  Minnesota Timberwolves    

# 8. Random forest!

In [23]:

def train_and_evaluate_rf_model(all_season_data, selected_features, n_estimators=100, max_depth=10):
    X_combined, y_combined = preprocess(all_season_data, selected_features)
    
    scaler_rf = StandardScaler() # standardize features 
    X_scaled = scaler_rf.fit_transform(X_combined)

    # training random forest
    rf_model = RandomForestRegressor(n_estimators=n_estimators, max_depth=max_depth, random_state=42)
    rf_model.fit(X_scaled, y_combined)
    
    # cross-validation
    cross_val_predictions = cross_val_predict(rf_model, X_scaled, y_combined, cv=5)
    avg_mse_rf = ((cross_val_predictions - y_combined) ** 2).mean()  # Mean Squared Error
    print(f"Random Forest Model Performance (Avg MSE): {avg_mse_rf}")
    
    #  predictions for each season
    season_predictions = {}
    for season, data_frame in merged_season_data.items():
        X_season = data_frame[selected_features]
        y_season = data_frame['Wins']
        X_season_scaled = scaler_rf.transform(X_season)
        season_predictions[season] = {
            'Team': data_frame['TEAM_NAME'],
            'Actual Wins': y_season,
            'Predicted Wins': rf_model.predict(X_season_scaled)
        }
    
    return rf_model, scaler_rf, season_predictions

# train/evaluate Random Forest
rf_model_combined, rf_scaler_combined, all_season_predictions = train_and_evaluate_rf_model(
    all_season_data, selected_features_combined, n_estimators=500, max_depth=10
)


for season, results in all_season_predictions.items():
    predictions_df = pd.DataFrame(results)
    print(f"\nPredictions for {season}:")
    print(predictions_df)


Random Forest Model Performance (Avg MSE): 10.720480646364452

Predictions for 2016-17:
                      Team  Actual Wins  Predicted Wins
0            Atlanta Hawks          2.0        3.261085
1           Boston Celtics          9.0        8.472349
2            Brooklyn Nets          NaN        0.000000
3        Charlotte Hornets          NaN        0.014000
4            Chicago Bulls          2.0        1.430333
5      Cleveland Cavaliers         13.0       10.163537
6         Dallas Mavericks          NaN        0.010000
7           Denver Nuggets          NaN        0.045509
8          Detroit Pistons          NaN        0.106000
9    Golden State Warriors          NaN        1.854057
10         Houston Rockets          6.0        6.212452
11          Indiana Pacers          0.0        0.546809
12             LA Clippers          NaN        1.544746
13      Los Angeles Lakers          NaN        0.026000
14       Memphis Grizzlies          2.0        1.692296
15              

# 8. Calculating RMSE for each season

In [27]:
def calculate_rmse_per_season(all_season_predictions):
    season_rmse = {}
    
    for season, results in all_season_predictions.items():
        actual_wins = results['Actual Wins']
        predicted_wins = results['Predicted Wins']

        actual_wins = actual_wins.fillna(0)  # replace NaN values w 0 in actual wins
        

        mse = mean_squared_error(actual_wins, predicted_wins)  
        rmse = np.sqrt(mse)
        
        season_rmse[season] = rmse
    
    return season_rmse


season_rmse = calculate_rmse_per_season(all_season_predictions)


for season, rmse in season_rmse.items():
    print(f"RMSE for {season}: {rmse}")

RMSE for 2016-17: 0.9694751469134324
RMSE for 2017-18: 0.9580102595473466
RMSE for 2018-19: 1.1660502925296938
RMSE for 2019-20: 1.3923958675846202
RMSE for 2020-21: 1.2133994715172538
RMSE for 2021-22: 1.0877619697068792
RMSE for 2022-23: 1.4888391745814535
RMSE for 2023-24: 1.4734461615425904


# 10. Averae RMSE for the random forest model

In [28]:
#average rmse of the model

avg_rmse = np.mean(list(season_rmse.values()))
print("Average RMSE across all seasons:", avg_rmse)

Average RMSE across all seasons: 1.2186722929904088


# 11. Predicting the 2025 playoffs!

In [29]:
current_year = season_data_frames['2024-25']
X_current = current_year[selected_features_combined]
X_current_scaled = rf_scaler_combined.transform(X_current)

current_year_predictions = rf_model_combined.predict(X_current_scaled)
current_year['Predicted_Wins_2025'] = current_year_predictions
print("Predictions for the current year (2025):")
print(current_year[['Team', 'Predicted_Wins_2025']])


Predictions for the current year (2025):
   Team  Predicted_Wins_2025
0   ATL             0.170010
1   BOS             7.248406
2   BKN             0.006000
3   CHO             0.006000
4   CHI             0.140000
5   CLE             7.022000
6   DAL             0.907509
7   DEN             8.798000
8   DET             0.883227
9   GSW             0.554726
10  HOU             6.640016
11  IND             2.040721
12  LAC             1.622125
13  LAL             2.207443
14  MEM             8.353902
15  MIA             0.443865
16  MIL             0.298594
17  MIN             1.618954
18  NOP             0.000000
19  NYK             2.999258
20  OKC             6.152399
21  ORL             0.330833
22  PHI             0.000000
23  PHX             1.357692
24  POR             0.006000
25  SAC             0.416513
26  SAS             0.004000
27  TOR             0.002000
28  UTA             0.000000
29  WAS             0.014000
